In [32]:
import sqlite3
import pandas as pd


con = sqlite3.connect('imdb_small_indexed.db') # коннект к бд
cur = con.cursor() # инициализация курсора

# Переменнные для передачи в запрос
genre1 = 1 # ID жанра от
genre2 = 10 # до
year1 = 2000 # Год от
year2 = 2005 # до

# Запрос
query = """select gt.genre_name,count(rc.id) as count, rc.role_type, t.premiered as Year
from titles t join crew c on t.title_id = c.title_id
    join role_categories rc on c.category = rc.id
    join film_genres fg on t.title_id = fg.title_id
    join genre_types gt on fg.genre_id = gt.id
where rc.id in (1, 2) and gt.id between ? and ? and t.premiered between ? and ?-- ? - Переменная
group by rc.role_type, gt.genre_name, t.premiered
order by gt.id asc
"""

cur.execute(query, (genre1, genre2, year1, year2)) # Выполнение запроса с переменными
a = cur.fetchall()

df = pd.read_sql_query(query, params= [genre1, genre2, year1, year2], con=con) # Передаем запрос в пандас
df = df.rename(columns={"genre_name": "Genre", "count": "Count", "role_type" : "Sex"}) # Переименуем столбцы
df = df.pivot_table(index=["Genre", "Year"], columns = "Sex", values = "Count") # Переформатируем таблицу
df = df.rename(columns={"actor": "Man", "actress" : "Woman"}) # Переименную столбцы в отформатированной таблице

#Считаем проценты
df["Man %"] =  round(df["Man"]/(df["Man"] + df["Woman"]) * 100, 1)
df["Woman %"] =  round(df["Woman"]/(df["Man"] + df["Woman"]) * 100, 1)

df


Sex                Man  Woman  Man %  Woman %
Genre       Year                             
Action      2000  2115    923   69.6     30.4
            2001  2365   1042   69.4     30.6
            2002  2237   1005   69.0     31.0
            2003  2280   1026   69.0     31.0
            2004  2407   1070   69.2     30.8
            2005  2373   1033   69.7     30.3
Adult       2000   129    204   38.7     61.3
            2001   139    232   37.5     62.5
            2002   149    221   40.3     59.7
            2003   164    307   34.8     65.2
            2004   213    294   42.0     58.0
            2005   184    364   33.6     66.4
Adventure   2000   805    457   63.8     36.2
            2001   994    532   65.1     34.9
            2002   957    566   62.8     37.2
            2003  1174    664   63.9     36.1
            2004  1004    551   64.6     35.4
            2005  1116    604   64.9     35.1
Animation   2000   839    636   56.9     43.1
            2001  1016    759   57.2     42.8
            2002   948    721   56.8     43.2
            2003  1184    898   56.9     43.1
            2004  1165    828   58.5     41.5
            2005  1170    943   55.4     44.6
Biography   2000   231    108   68.1     31.9
            2001   212    109   66.0     34.0
            2002   177     98   64.4     35.6
            2003   185    102   64.5     35.5
            2004   251    135   65.0     35.0
            2005   335    137   71.0     29.0
Comedy      2000  4684   3021   60.8     39.2
            2001  4926   3168   60.9     39.1
            2002  5316   3337   61.4     38.6
            2003  6083   3658   62.4     37.6
            2004  6157   3845   61.6     38.4
            2005  7062   4466   61.3     38.7
Crime       2000  1623    747   68.5     31.5
            2001  1565    771   67.0     33.0
            2002  1620    804   66.8     33.2
            2003  1643    780   67.8     32.2
            2004  1592    759   67.7     32.3
            2005  1741    852   67.1     32.9
Documentary 2000   724    313   69.8     30.2
            2001   775    319   70.8     29.2
            2002   890    395   69.3     30.7
            2003  1187    504   70.2     29.8
            2004  1326    520   71.8     28.2
            2005  1463    564   72.2     27.8
Drama       2000  6657   4322   60.6     39.4
            2001  7264   4918   59.6     40.4
            2002  7541   5250   59.0     41.0
            2003  7880   5440   59.2     40.8
            2004  8493   5882   59.1     40.9
            2005  9469   6626   58.8     41.2
Family      2000   938    644   59.3     40.7
            2001  1074    733   59.4     40.6
            2002  1129    839   57.4     42.6
            2003  1091    746   59.4     40.6
            2004   948    711   57.1     42.9
            2005   982    754   56.6     43.4